# Spotify playlist creator

Read music directory

In [ ]:
def is_mp3(n: str) -> bool:
    return n.lower()[-4:] == ".mp3"

def remove_ext(n: str) -> str:
    return n[:-4]

In [ ]:
MUSIC_PATH: str = "/mnt/d/Music"

files: dict[str, list[str]] = {}

In [ ]:
import os

def sanitize(base_path: str, path: str) -> str:
    f = path[len(base_path)+1:]
    return f.replace(" ", "").replace("-","").replace("/","-")


def get_files(path:str):
    for j, _, filenames in os.walk(path):
        sanitized: str = sanitize(base_path=path, path=j)

        mp3s = [remove_ext(m) for m in filenames if is_mp3(m)]
        if mp3s:
            files[sanitized] = mp3s

In [ ]:
get_files(MUSIC_PATH)

## Create playlists - musics

In [ ]:
import spotify

**Get spotify token**

https://developer.spotify.com/console/get-search-item/?q=&type=&market=&limit=&offset=&include_external=

Click in ***Get token***

Wehn the pop up show up, select:

- playlist-modify-private
- playlist-read-private
- user-library-modify
- user-library-read

Then, copy the OAuth Token

In [ ]:
token: str = ""

**Get User**

In [ ]:
user_id: str | None = spotify.user_id(jwt=token)
if not user_id:
    raise Exception("fail to get user")

In [ ]:
playlists_success: list[str] = []
musics_success: list[str] = []
worked: list[str] = []

for playlist_name in files:
    musics: list[str] = files[playlist_name]

    # create playlist if not exists
    playlist_id: str | None = spotify.create_playlist(name=playlist_name,user_id=user_id, jwt=token)
    if not playlist_id:
        print(f"fail to create playlist: {playlist_name}")
        continue

    playlists_success.append(playlist_name)

    # search musics
    for music_name in musics:
        track_id: str | None = spotify.search_track(name=music_name.lower(), jwt=token)
        if not track_id:
            logname: str = f"{playlist_name}/{music_name}"
            print(f"fail to get music: {logname}")
            continue
        musics_success.append(music_name)
        
        # add music to playlist
        worked = spotify.add_track_playlist(
            playlist_id=playlist_id, tracks_ids=[track_id], jwt=token)
        if not worked:
            logname: str = f"{playlist_name}/{music_name}"
            print(f"fail to add music: {music_name} to {playlist_name}")
